<a href="https://colab.research.google.com/github/ijortin/CS-4485-Virtual-TA/blob/prediction/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paackage installation

In [6]:
!pip install tflearn
!pip install nltk
!pip install numpy
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement punkt (from versions: none)
ERROR: No matching distribution found for punkt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Imports

In [7]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
from google.colab import files
uploaded = files.upload()
tempData = next(iter(uploaded.values()))

Saving intents.json to intents.json


Load in data

In [9]:
data = json.loads(tempData.decode())

In [12]:
try:
  with open("data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)

except:
  words = []
  labels = []
  docs_x = []
  docs_y = []

  for intent in data["intents"]:
      for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

      if intent["tag"] not in labels:
        labels.append(intent["tag"])
  words = [stemmer.stem(w.lower()) for w in words if w != "?"]
  words = sorted(list(set(words)))

  labels = sorted(labels)

  training = []
  output = []

  out_empty = [0 for _ in range(len(labels))]

  for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

  training = numpy.array(training)
  output = numpy.array(output)

Train Model

In [46]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 3)
#net = tflearn.fully_connected(net, 3)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [47]:
model.fit(training, output, n_epoch=1000, batch_size=3, show_metric= True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 0.02682 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.02682 - acc: 1.0000 -- iter: 09/10
Training Step: 4000  | total loss: 0.02611 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.02611 - acc: 1.0000 -- iter: 10/10
--


Predictions

In [ ]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1
  
  return numpy.array(bag)

In [52]:
def chat():
  print("Start talking with the bot (type quit to stop)!")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break

    results = model.predict([bag_of_words(inp,words)])
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']

    print(random.choice(responses))

In [53]:
chat()

Start talking with the bot (type quit to stop)!
You: hi
Hi there, how can I help?
You: hello
Good to see you again
You: howdy
Good to see you again
You: goodbye
Bye! Come back again soon
You: see you later
Bye! Come back again soon
You: thanks
Any Time!
You: help
My pleasure
You: quit
